<a href="https://colab.research.google.com/github/SaurabhSRP/Recommnedation-system-techniques/blob/main/Recommendation_system_3_Nearest_Neighbor_%26_KNN_for_collaborative_filtering_for_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Nearest Neighbor item based Collaborative Filtering**


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies_df=pd.read_csv("/content/drive/MyDrive/datasets/MovieRecom/KNN Movie Recommendation/movies.csv",usecols=['movieId','title'],dtype={'movieId':'int32','title':'str'})
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [ ]:
rating_df=pd.read_csv("/content/drive/MyDrive/datasets/MovieRecom/KNN Movie Recommendation/ratings.csv",usecols=['userId','movieId','rating'],dtype={'userId':'int32','movieId':'int32','rating':'float32'})
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
df=pd.merge(rating_df,movies_df,on='movieId')

In [ ]:
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [ ]:
combine_movie_rating=df.dropna(axis=0,subset=['title'])

In [ ]:
movie_ratingCount=(combine_movie_rating.groupby(['title'])['rating'].count())

In [ ]:
movie_ratingCount

title
'71 (2014)                                    1
'Hellboy': The Seeds of Creation (2004)       1
'Round Midnight (1986)                        2
'Salem's Lot (2004)                           1
'Til There Was You (1997)                     2
                                             ..
eXistenZ (1999)                              22
xXx (2002)                                   24
xXx: State of the Union (2005)                5
¡Three Amigos! (1986)                        26
À nous la liberté (Freedom for Us) (1931)     1
Name: rating, Length: 9719, dtype: int64

In [ ]:
movie_ratingCount=movie_ratingCount.reset_index()

In [ ]:
movie_ratingCount

,title,rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [ ]:
movie_ratingCount=movie_ratingCount.rename(columns={'rating':'totalRatingCount'})

In [ ]:
movie_ratingCount

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [ ]:
rating_with_totalRatingCount=combine_movie_rating.merge(movie_ratingCount,left_on='title',right_on='title',how='left')

In [ ]:
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [ ]:
pd.set_option('display.float_format',lambda x:'%.3f'%x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [ ]:
popularity_threshold=50 
rating_popular_movie=rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,5,1,4.000,Toy Story (1995),215
2,7,1,4.500,Toy Story (1995),215
3,15,1,2.500,Toy Story (1995),215
4,17,1,4.500,Toy Story (1995),215


In [ ]:
rating_popular_movie.shape

(41362, 5)

In [ ]:
##lets create a pivot table
movie_feature_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_feature_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [ ]:
movie_feature_df.values

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 3. , 0. , 4.5],
       ...,
       [5. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 3. , 0. , ..., 0. , 0. , 3.5],
       [0. , 0. , 0. , ..., 3. , 0. , 4. ]], dtype=float32)

In [ ]:
movie_feature_df.shape

(450, 606)

In [ ]:
##lets convert this pivot table into a matrix 
from scipy.sparse import csr_matrix

movie_feature_df_matrix = csr_matrix(movie_feature_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movie_feature_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
movie_feature_df[movie_feature_df.shape[0]]

title
10 Things I Hate About You (1999)   0.000
12 Angry Men (1957)                 0.000
2001: A Space Odyssey (1968)        0.000
28 Days Later (2002)                0.000
300 (2007)                          0.000
                                     ... 
X2: X-Men United (2003)             0.000
You've Got Mail (1998)              0.000
Young Frankenstein (1974)           0.000
Zombieland (2009)                   0.000
Zoolander (2001)                    0.000
Name: 450, Length: 450, dtype: float32

In [ ]:
query_index=np.random.choice(movie_feature_df.shape[0])
print(query_index)

338


In [ ]:
movie_feature_df.iloc[query_index,:]

userId
1     4.000
2     0.000
3     0.000
4     0.000
5     0.000
       ... 
606   4.000
607   5.000
608   4.500
609   0.000
610   5.000
Name: Saving Private Ryan (1998), Length: 606, dtype: float32

In [ ]:
movie_feature_df.iloc[query_index,:].values

array([4. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 0. ,
       0. , 3.5, 0. , 4.5, 4. , 0. , 0. , 0. , 0. , 0. , 4. , 5. , 0. ,
       0. , 3.5, 5. , 0. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2. , 5. , 0. , 0. , 5. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. ,
       0. , 0. , 0. , 5. , 0. , 5. , 0. , 0. , 4. , 3.5, 4. , 4. , 4.5,
       4.5, 4. , 0. , 0. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. ,
       0. , 0. , 3.5, 5. , 0. , 0. , 3.5, 0. , 0. , 0. , 0. , 4.5, 0. ,
       5. , 0. , 5. , 4. , 5. , 0. , 0. , 4. , 0. , 0. , 4. , 0. , 5. ,
       0. , 0. , 0. , 0. , 3.5, 0. , 5. , 0. , 0. , 5. , 0. , 0. , 4. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4.5, 0. , 2.5,
       0. , 0. , 0. , 4. , 0. , 4. , 0. , 0. , 5. , 4. , 0. , 0. , 3.5,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4. , 0. , 0. ,
       0. , 0. , 5. , 0. , 0. , 0. , 0. , 3. , 4.5, 3.5, 0. , 5. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 0. , 0. , 3. , 5.

In [ ]:
movie_feature_df.iloc[query_index,:].values.reshape(1,-1)

array([[4. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 0. ,
        0. , 3.5, 0. , 4.5, 4. , 0. , 0. , 0. , 0. , 0. , 4. , 5. , 0. ,
        0. , 3.5, 5. , 0. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 2. , 5. , 0. , 0. , 5. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. ,
        0. , 0. , 0. , 5. , 0. , 5. , 0. , 0. , 4. , 3.5, 4. , 4. , 4.5,
        4.5, 4. , 0. , 0. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. ,
        0. , 0. , 3.5, 5. , 0. , 0. , 3.5, 0. , 0. , 0. , 0. , 4.5, 0. ,
        5. , 0. , 5. , 4. , 5. , 0. , 0. , 4. , 0. , 0. , 4. , 0. , 5. ,
        0. , 0. , 0. , 0. , 3.5, 0. , 5. , 0. , 0. , 5. , 0. , 0. , 4. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4.5, 0. , 2.5,
        0. , 0. , 0. , 4. , 0. , 4. , 0. , 0. , 5. , 4. , 0. , 0. , 3.5,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4. , 0. , 0. ,
        0. , 0. , 5. , 0. , 0. , 0. , 0. , 3. , 4.5, 3.5, 0. , 5. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 0. 

In [ ]:
distances,indices=model_knn.kneighbors(movie_feature_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6) ###neightbors decide no of recommendation you need ##here five reconmmendation will be given along with the random selected movie name which makes it 6


In [ ]:
distances

array([[0.        , 0.32038516, 0.3837443 , 0.38452482, 0.39380217,
        0.40724707]], dtype=float32)

In [ ]:
distances.flatten()

array([0.        , 0.32038516, 0.3837443 , 0.38452482, 0.39380217,
       0.40724707], dtype=float32)

In [ ]:
indices

array([[338, 264, 358, 386, 385, 384]])

In [ ]:
for i in range(0,len(distances.flatten())):
  if i==0:
    print('Recommendations for {0}:\n'.format(movie_feature_df.index[query_index]))
  else:
    print('{0}:{1}, with distance of {2}'.format(i,movie_feature_df.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for Saving Private Ryan (1998):

1:Matrix, The (1999), with distance of 0.3203851580619812
2:Sixth Sense, The (1999), with distance of 0.3837442994117737
3:Star Wars: Episode VI - Return of the Jedi (1983), with distance of 0.3845248222351074
4:Star Wars: Episode V - The Empire Strikes Back (1980), with distance of 0.3938021659851074
5:Star Wars: Episode IV - A New Hope (1977), with distance of 0.40724706649780273
